# Day 1

## Part 1

At first I tried to use `map read` on the lines of the input, but that generates a runtime error. Positive frequencies are encoded with a leading `+` sign and Haskell refuses to parse those as one might expect. So here, we define a custom `parseInt` function that pulls off the leading `+` signs before passing the rest onto `read`.

In [1]:
parseInt :: String -> Int
parseInt ('+':rest) = read rest
parseInt str = read str

With that out of the way, we compute the resulting frequency after passing through all of the additions and subtracts by just using the `sum` function on the parsed input.

In [2]:
resultingFrequency :: String -> Int
resultingFrequency = sum . map parseInt . lines

In [3]:
readFile "./inputs/1.txt" >>= print . resultingFrequency

500

## Part 2

In [4]:
import qualified Data.Set as Set

intermediateFrequencies :: [Int] -> [Int]
intermediateFrequencies xs = 0 : f (0, xs)
  where
    f (currentFreq, []) = []
    f (currentFreq, (x:xs)) = let newFreq = currentFreq + x in newFreq : f (newFreq, xs)

firstDuplicate :: (Ord a) => [a] -> a
firstDuplicate xs = f Set.empty xs
  where
    f visited (x:xs) = if x `Set.member` visited
                         then x
                         else f (Set.insert x visited) xs

firstDuplicateFrequency :: String -> Int
firstDuplicateFrequency = firstDuplicate . intermediateFrequencies . cycle . map parseInt . lines

In [5]:
readFile "./inputs/1.txt" >>= print . firstDuplicateFrequency

709

# Day 2

## Part 1

There's a function `group` in `Data.List` that groups together subsequences of equal elements in a list.

In [6]:
import Data.List (group)
group "Mississippi"

["M","i","ss","i","ss","i","pp","i"]

If we apply this to a box ID after we sort its letters we'll get something like this.

In [7]:
let boxId = "evsialkqydurohxqpwbcugtjmh"

import Data.List (sort)
group . sort $ boxId

["a","b","c","d","e","g","hh","i","j","k","l","m","o","p","qq","r","s","t","uu","v","w","x","y"]

Now we can just test if there are any elements of this list that contain exactly 2 characters and test if there are any that contain exactly 3 elements.

In [8]:
any (\x -> length x == 2) . group . sort $ boxId

True

In [9]:
any (\x -> length x == 3) . group . sort $ boxId

False

Putting it all together:

In [10]:
checksum :: [String] -> Int
checksum xs = numIdsWithRepeatedChars 2 * numIdsWithRepeatedChars 3
  where
    numIdsWithRepeatedChars n = length . filter (any (\x -> length x == n) . group . sort) $ xs

In [11]:
day2Input <- readFile "./inputs/2.txt"
checksum . lines $ day2Input

6448

## Part 2

Naively, we can test each string against every other string to count the number of differing characters. I want a function that, given a list, returns a list of pairs of elements.

In [12]:
pairs :: [a] -> [(a,a)]
pairs (x:xs) = map (\y -> (x, y)) xs ++ pairs xs
pairs [] = []

For example,

In [13]:
pairs [1,2,3,4]

[(1,2),(1,3),(1,4),(2,3),(2,4),(3,4)]

Now that we can compute all of the pairs on box IDs we need to compare, we a way to calculate if two box IDs differ by exactly one character.

In [14]:
editDistance :: Eq a => [a] -> [a] -> Int
editDistance xs ys = length . filter (\(x, y) -> x /= y) $ zip xs ys

In [15]:
editDistance "abc" "xyz"

3

In [16]:
editDistance "abc" "azc"

1

Now we can find the unique pair of box IDs whose `editDistance` is 1. But after that, we'll need to construct the string composed of the letters shared between the two.

In [17]:
sharedSubsequence :: Eq a => [a] -> [a] -> [a]
sharedSubsequence xs ys = map fst . filter (\(x, y) -> x == y) $ zip xs ys

In [18]:
sharedSubsequence "abcde" "abzde"

"abde"

Finally,

In [19]:
import Data.List (find)

commonLettersBetweenCorrectBoxIds :: [String] -> Maybe String
commonLettersBetweenCorrectBoxIds xs = uncurry sharedSubsequence <$> pair
  where
    pair = find ((== 1) . uncurry editDistance) . pairs $ xs

In [20]:
commonLettersBetweenCorrectBoxIds . lines $ day2Input

Just "evsialkqyiurohzpwucngttmf"

# Day 3

## Part 1

The first challenge is parsing the input format. The input contains "claims" formatted like this:

```
#1 @ 662,777: 18x27
#2 @ 893,985: 13x10
#3 @ 199,328: 16x16
...
```

For Advent of Code projects in the past, I've used parser combinator libraries Parsec and Attoparsec for parsing even simple things like this. I think this time I'll just try to use a custom `Read` instance.

In [21]:
data FabricClaim = FabricClaim { identifier :: Int
                               , position :: (Int, Int)
                               , size :: (Int, Int) } deriving (Show, Eq)

In [22]:
import Data.Char (isDigit)

instance Read FabricClaim where
  readsPrec _ input =
    let ('#':rest1) = input
        (claimId, rest2) = span isDigit rest1
        (' ' : '@' : ' ' : rest3) = rest2
        (positionX, rest4) = span isDigit rest3
        (',':rest5) = rest4
        (positionY, rest6) = span isDigit rest5
        (':' : ' ' : rest7) = rest6
        (width, rest8) = span isDigit rest7
        ('x' : rest9) = rest8
        (height, rest10) = span isDigit rest9
        in
    [(FabricClaim (read claimId) (read positionX, read positionY) (read width, read height), rest10)]

In [23]:
read "#1 @ 662,777: 18x27" :: FabricClaim

FabricClaim {identifier = 1, position = (662,777), size = (18,27)}

In [24]:
read "[#1 @ 662,777: 18x27,#2 @ 893,985: 13x10]" :: [FabricClaim]

[FabricClaim {identifier = 1, position = (662,777), size = (18,27)},FabricClaim {identifier = 2, position = (893,985), size = (13,10)}]

I want to fold over the list of claims. I'll need to keep track of the total area claimed so far by at least one elf as well as the total area claimed by more than one elf. Naturally, I'll want to convert every claim to a set of coordinates.

In [25]:
getCoords :: FabricClaim -> Set.Set (Int, Int)
getCoords (FabricClaim claimdId (startX, startY) (w, h)) =
  Set.fromList [(x, y) | x <- [startX..(startX + w - 1)],
                         y <- [startY..(startY + h - 1)]]

In [26]:
getCoords $ FabricClaim 0 (1,2) (3,4)

fromList [(1,2),(1,3),(1,4),(1,5),(2,2),(2,3),(2,4),(2,5),(3,2),(3,3),(3,4),(3,5)]

Thus,

In [27]:
import Data.List (foldl')

areaInContention :: [FabricClaim] -> Set.Set (Int, Int)
areaInContention claims =
  let f (contended, claimed) claim = (Set.union contended . Set.intersection claimed $ getCoords claim,
                                      Set.union claimed $ getCoords claim)
    in
      fst $ foldl' f (Set.empty, Set.empty) claims               

In [28]:
day3Input <- readFile "./inputs/3.txt"
print . length . areaInContention . map read . lines $ day3Input

120408

## Part 2

Now we want to find the unique claim that intersects no other claim. For this, we can iterate over every claim and check to see if its coordinate set is disjoint with the union of the coordinates of the other claims.

In [29]:
nonOverlappingClaim :: [FabricClaim] -> Maybe FabricClaim
nonOverlappingClaim xs =
  let precomputedCoords = zip xs $ map getCoords xs
      nonOverlapping :: (FabricClaim, Set.Set (Int, Int)) -> Bool
      nonOverlapping x@(claim, coords) = all (Set.disjoint coords . snd) $ filter (/= x) precomputedCoords
    in
      fst <$> find nonOverlapping precomputedCoords

In [30]:
print . nonOverlappingClaim . map read . lines $ day3Input

Just (FabricClaim {identifier = 1276, position = (568,851), size = (14,23)})

# Day 4

## Part 1

Let's set up some data structures to represent the log data. The log entries in the input look like this:

```
[1518-11-01 00:00] Guard #10 begins shift
[1518-11-01 00:05] falls asleep
[1518-11-01 00:25] wakes up
```

In [31]:
import Data.Time (UTCTime)

data GuardEvent = GuardBeginsShift Int | GuardFallsAsleep | GuardWakesUp deriving Show
data LogEntry = LogEntry UTCTime GuardEvent deriving Show

Let's see if the `Read` instance of `UTCTime` will work for the format that appears in the input.

In [32]:
read "1518-11-03 00:29" :: UTCTime

Prelude.read: no parse

In [33]:
read "1518-11-03 00:29:00" :: UTCTime

1518-11-03 00:29:00 UTC

So it looks like we'll have to append a `":00"` before we try to `read` the timestamp. No big deal. Let's try to parse the log entries.

In [34]:
import Text.ParserCombinators.ReadP (ReadP, char, munch, satisfy, (<++), string)
import qualified Text.ParserCombinators.ReadPrec as ReadPrec
import qualified Text.Read as Read

guardEvent :: ReadP GuardEvent
guardEvent =
  let guardBeginsShift = GuardBeginsShift <$> (string "Guard #" *> (read <$> munch isDigit))
                                          <* string " begins shift"
      guardFallsAsleep = GuardFallsAsleep <$ string "falls asleep"
      guardWakesUp = GuardWakesUp <$ string "wakes up"
    in guardBeginsShift <++ guardFallsAsleep <++ guardWakesUp

instance Read GuardEvent where
  readPrec = ReadPrec.lift guardEvent

instance Read LogEntry where
  readPrec =
    let timestampString = (\x -> read $ x ++ ":00") <$> (char '[' *> munch (/= ']')) <* char ']'
      in ReadPrec.lift $ LogEntry <$> (timestampString <* char ' ') <*> guardEvent

In [35]:
read "[1518-11-01 00:00] Guard #10 begins shift" :: LogEntry

LogEntry 1518-11-01 00:00:00 UTC (GuardBeginsShift 10)

In [36]:
read "[1518-11-01 00:05] falls asleep" :: LogEntry

LogEntry 1518-11-01 00:05:00 UTC GuardFallsAsleep

In [37]:
read "[1518-11-01 00:25] wakes up" :: LogEntry

LogEntry 1518-11-01 00:25:00 UTC GuardWakesUp